In [1]:
# Needed libraries 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.key_input import KeyInput
from selenium.webdriver.common.actions.pointer_input import PointerInput

# System requiremenmts 
import time
import random
import datetime as dt

# Data Management 
import pandas as pd

# Web scrapping functions 
import src.web_minning as wm

In [2]:
# Parameters 

CITY = "cdmx"
TAGS_LIST = wm.city_hashtags(CITY)
MAX_DEPTH = 5 # Iteration loop over each initial video
#MAX_SCROLLING = #

RAW_DATA = "./data/raw/"
ROOT_FILE = "root_links.csv"
DAY_FILE = f"day_file_{CITY}_2026-01-05.csv"
DAYS_FRESHNESS = 300

# Mexico City (CDMX) bounding box
# Format: (min_lat, min_lon, max_lat, max_lon)
CITY_BBOX = (
    19.0000,   # min_lat (south)
    -99.3600,  # min_lon (west)
    19.6000,   # max_lat (north)
    -98.9400   # max_lon (east)
)


### Data Cleaning and mapping 

In [13]:
def improve_label(label, city):
    label = (label or "").strip()
    city = (city or "").strip()

    label_l = label.lower()
    city_l = city.lower()

    # Validate city in label
    if city_l and city_l not in label_l:
        label = f"{label}, {city.strip().title()}"

    # Add more rules [DEBUG]
    return label


def get_locations_from_text(desc_text, city):
    desc_text = (desc_text or "").strip()
    add_label = ""
    loc_osm_api = {"found": False}

    if not desc_text:
        return add_label, loc_osm_api

    try:
        exact_add = wm.extract_addresses(desc_text, top_k=1, min_score=4.5)
    except Exception:
        exact_add = []

    if exact_add:
        add_label = improve_label(exact_add[0].text, city)
        try:
            loc_osm_api = wm.geocode_osm(add_label)
        except Exception:
            loc_osm_api = {"found": False}
        return add_label, loc_osm_api

    try:
        places = wm.detect_places(desc_text)
    except Exception:
        places = []

    if places:
        add_label = improve_label(places[0].text, city)
        try:
            loc_osm_api = wm.geocode_osm(add_label)
        except Exception:
            loc_osm_api = {"found": False}

    return add_label, loc_osm_api


In [3]:
import logging
from typing import Any, Dict, Tuple


logger = logging.getLogger(__name__)


def get_best_location(
    d: Any,
    desc_text: str,
    city: str,
) -> Tuple[bool, float, float, str, str]:
    """
    Resolve the best location from text using OSM-backed extraction.

    Returns:
        (found, lat, lon, osm_display_name, label_used)
    """
    found = False
    lat = 0.0
    lon = 0.0
    osm_name_label = ""
    add_label_used = ""

    def _parse_osm_result(
        location_label: str,
        osm_result: Dict[str, Any],
    ) -> Tuple[bool, float, float, str, str]:
        if not isinstance(osm_result, dict):
            logger.warning("OSM result is not a dict: %s", type(osm_result))
            return False, 0.0, 0.0, "", ""

        if not osm_result.get("found", False):
            return False, 0.0, 0.0, "", location_label or ""

        try:
            lat_val = float(osm_result.get("lat", 0.0))
            lon_val = float(osm_result.get("lon", 0.0))
        except (TypeError, ValueError):
            logger.exception("Invalid lat/lon in OSM result: %s", osm_result)
            return False, 0.0, 0.0, "", location_label or ""

        display_name = str(osm_result.get("display_name", "") or "")
        return True, lat_val, lon_val, display_name, location_label or ""

    #logger.info("Resolving location. city=%s desc_len=%s", city, len(desc_text or ""))

    try:
        location_label, osm_result = get_locations_from_text(desc_text, city)
        #logger.info(
        #    "get_locations_from_text returned found=%s label=%r",
        #    bool(isinstance(osm_result, dict) and osm_result.get("found")),
        #    location_label,
        #)

        found, lat, lon, osm_name_label, add_label_used = _parse_osm_result(
            location_label, osm_result
        )

        if not found:
            logger.info("No location found for city=%s", city)
            return found, lat, lon, osm_name_label, add_label_used

        #logger.info(
        #    "Location resolved: lat=%s lon=%s display_name=%r label_used=%r",
        ##    lat,
         #   lon,
         #   osm_name_label,
         #   add_label_used,
        #)
        return found, lat, lon, osm_name_label, add_label_used

    except Exception:
        #logger.exception("Failed to resolve location. city=%s", city)
        return False, 0.0, 0.0, "", ""


In [17]:
location_label,loc_osm_api =  get_locations_from_text(desc_text, CITY)

In [22]:
found, lat, lon, osm_name_label, add_label_used = get_best_location(d,desc_text,CITY)

In [4]:
sites_df = pd.read_csv(RAW_DATA + DAY_FILE)
sites_df.description = sites_df.description.fillna('')

In [6]:
sites_df.loc[3,['description']].values[0]

''

In [23]:
sites_df.tail(7)#.query(""" (ind_found == True) and (osm_name_label != 'Ciudad de México, México') """)

,href,description,date,hashtag_label,level,long_description,date_long,add_label_used,ind_found,lat,lon,osm_name_label
2876,https://www.tiktok.com/@revistapicnic/video/75...,Si aún no conoces,2025-12-18,#bazar #cdmx,0,NaN,2025-12-18,NaN,False,0.000000,0.000000,NaN
2877,https://www.tiktok.com/@renosanntos/video/7583...,Que hacer en cdmx 💿 con $100 pesitos?!!,2025-12-14,#bazarcdmx,0,Actividad en CDMX con $100 Pesos: Bazar Recome...,2025-12-14,NaN,False,0.000000,0.000000,NaN
2878,https://www.tiktok.com/@casaoblivion.bazar/vid...,LA PRIMERA EDICIÓN DEL 2026 😍✨,Hace 2 d,#bazarcdmx,0,Casa Oblivion 2026: Bazar Único en Narvarte\nD...,2026-01-06,cdmx 🤑🇲🇽,True,19.320772,-99.151468,"Ciudad de México, México"
2879,https://www.tiktok.com/@merchadamx/video/75898...,La última y nos vamos! 🫰🏻,2025-12-30,#bazarcdmx,0,Bazar de Chabacano: ¡No te lo pierdas!\n¡Únete...,2025-12-30,dejarnos ser parte de tu agenda citadina ❤️ Es...,False,0.000000,0.000000,NaN
2880,https://www.tiktok.com/@tan_lunna/video/754640...,Plan en Fin de semana en CDMX,2025-9-4,#bazarcdmx,0,Disfruta del Filomena Market en CDMX este fin ...,2025-09-04,NaN,False,0.000000,0.000000,NaN
2881,https://www.tiktok.com/@nylagame/video/7591132...,Feria del juguete San Felipe de Jesús o san fe...,Hace 2 d,#bazarcdmx,0,Feria del Juguete San Felipe de Jesús: Precios...,2026-01-06,NaN,False,0.000000,0.000000,NaN
2882,https://www.tiktok.com/@stefaniroux17/video/75...,"Feria y Romería de la Ciudad de México, que pr...",Hace 3 d,#bazarcdmx,0,NaN,2026-01-06,la Ciudad de México 🇲🇽,True,19.290484,-99.657563,"La Ciudad de México, Calle Nicolás Bravo, Tolu..."


In [24]:
sites_df.loc[2878, "long_description"]

'Casa Oblivion 2026: Bazar Único en Narvarte\nDisfruta de la primera edición de Casa Oblivion en 2026. ¡Entrada gratuita y marcas favoritas! Te esperamos el 24 y 25 de enero. #dondeircdmx #bazarescdmx #plancdmx\nKeywords: Casa Oblivion 2026 Narvarte bazar gratuito, marcas favoritas bazar, eventos en CDMX enero 2026, moda retro y2k, ropa 2nd hand en CDMX, bazar de accesorios minimalistas, productos únicos de Pinterest, Castillo Rebsamen bazar, eventos de moda en México, bazar urbano en Narvarte\n'

In [26]:
text = sites_df.loc[2878, "long_description"]
events = wm.extract_event_occurrences(text)
events[0]


{'start': datetime.datetime(2026, 1, 25, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Mexico_City')),
 'end': None,
 'date_text': '25 de enero',
 'time_text': None,
 'span': (0, 329),
 'matched_text': 'Casa Oblivion 2026: Bazar Único en Narvarte\nDisfruta de la primera edición de Casa Oblivion en 2026. ¡Entrada gratuita y marcas favoritas! Te esperamos el 24 y 25 de enero. #dondeircdmx #bazarescdmx #plancdmx\nKeywords: Casa Oblivion 2026 Narvarte bazar gratuito, marcas favoritas bazar, eventos en CDMX enero 2026, moda retro y2k',
 'confidence': 0.718,
 'labels': ('event_hint', 'keyword', 'time_missing')}

In [ ]:
start_dt = events[0]['start']
end_dt = events[0]['end']


In [ ]:
type(events[0])

dict

In [ ]:
desc_results = []

n = len(sites_df)
k = 0

for h, d,desc_text in zip(sites_df.href.to_list(),sites_df.description.to_list(),sites_df.long_description.to_list()):
    
    time.sleep(random.uniform(0.5,1))

    found, lat, lon, osm_name_label, add_label_used = get_best_location(d,desc_text,CITY)

    desc_results.append(
        {
        "href": h,
        "long_description": desc_text,
        "add_label_used": add_label_used,
        "ind_found":found, 
        "lat":lat,
        "lon":lon, 
        "osm_name_label":osm_name_label

    }
    )

    k += 1

    print(f"Process ended for {h} - {k}  /{n}", end = "\r")

geo_desc_df = pd.DataFrame(desc_results)


Process ended for https://www.tiktok.com/@moniksmr/video/7545214454246444306
Process ended for https://www.tiktok.com/@nylagame/video/7591132225698090260
Process ended for https://www.tiktok.com/@elhilonegro/video/7557493636921888007
Process ended for https://www.tiktok.com/@moniksmr/video/7537785749467401490
Process ended for https://www.tiktok.com/@sally_qiiu/video/7591718836726975765
Process ended for https://www.tiktok.com/@dondeir/video/7591003063272410376
Process ended for https://www.tiktok.com/@undiacondya/video/7535855035767459079
Process ended for https://www.tiktok.com/@brenc.toys/video/7584180403234262279
Process ended for https://www.tiktok.com/@undiacondya/video/7515511709621079314
Process ended for https://www.tiktok.com/@soyhali/video/7591275674783042837
Process ended for https://www.tiktok.com/@moniksmr/video/7572281843647630610
Process ended for https://www.tiktok.com/@chefandresgallegos/video/7591279840234458375
Process ended for https://www.tiktok.com/@adondeconsaek

ValueError: If using all scalar values, you must pass an index

In [ ]:
georef_df = pd.DataFrame(
    {
        "href": h,
        "long_description": desc_text,
        "add_label_used": add_label_used,
        "ind_found":found, 
        "lat":lat,
        "lon":lon, 
        "osm_name_label":osm_name_label

    }
)

### Data Minning 

In [3]:
results = []


for ht in TAGS_LIST:


    try:
        # Invoke driver object 

        driver = webdriver.Chrome()
        wait = WebDriverWait(driver, 15)
        actions = ActionChains(driver)
        keyboard = KeyInput("keyboard")
        actions_b = ActionBuilder(driver, keyboard=keyboard)


        # Open Main page
        driver.get("https://www.tiktok.com")


        # Look for search icon 
        search_icon = wait.until(EC.presence_of_element_located((
            By.XPATH, "//*[@role='searchbox']"
        )))

        # Put mose in search bar 
        search_icon.click()

        # Input labels 
        actions.send_keys(ht)
        time.sleep(random.uniform(0.05, 0.15))
        actions.perform()

        # Enter to look 
        actions.send_keys("\n")
        actions.perform()

        time.sleep(random.uniform(1.5, 4.5))


        # Getting all inmediate results 
        items = wm.scroll_until_no_new_items(driver)
        wm.scroll_through_elements(driver, items)
        items = driver.find_elements(By.CSS_SELECTOR, "div[id^='grid-item-container-']")

        # Get data from results

        for container in items:
            try:
                href = container.find_element(
                    By.CSS_SELECTOR, "a[href*='/video/']"
                ).get_attribute("href")

                description = container.find_element(
                    By.CSS_SELECTOR, "[data-e2e='new-desc-span']"
                ).text.strip()

                date = container.find_element(
                    By.CSS_SELECTOR, "div[class*='DivTimeTag']"
                ).text.strip()

                results.append({
                    "href": href,
                    "description": description,
                    "date": date,
                    "hashtag_label": ht
                })

            except Exception:
                continue

        print(f"Links search for {ht} ended.")

        driver.quit()

    except:
        driver.quit()



Links search for #dondeircdmx ended.
Links search for #quehacercdmx ended.
Links search for #places #cdmx ended.
Links search for #placescdmx ended.
Links search for #lugaresimperdibles #cdmx ended.
Links search for #lugaressecretoscdmx ended.
Links search for #postres #cdmx ended.
Links search for #evento #cdmx ended.
Links search for #eventoscdmx ended.
Links search for #agendacdmx ended.
Links search for #conciertocdmx ended.
Links search for #festivalcdmx ended.


In [ ]:
df_raw = pd.DataFrame(results)
df_raw['level'] = 0




In [5]:
df_raw.to_csv(RAW_DATA + ROOT_FILE, index = False)

In [ ]:
root_list = df_raw.href.drop_duplicates().to_list()
caption_list = df_raw.description.drop_duplicates().to_list()

desc_results = []

for h, d in zip(root_list, caption_list):

    try:

        driver = webdriver.Chrome()
        wait = WebDriverWait(driver, 15)
        actions = ActionChains(driver)
        keyboard = KeyInput("keyboard")
        actions_b = ActionBuilder(driver, keyboard=keyboard)

        driver.get(h)

        time.sleep(random.uniform(2.5,3.5))

        more_buton = driver.find_element(
            By.CSS_SELECTOR,
            "button[class*='ButtonExpand']"
        )

        more_buton.click()

        time.sleep(random.uniform(2.5,3.5))


        # Date Extraction 
        try:
            raw = driver.find_element(
            By.CSS_SELECTOR,
            "span[class*='StyledTUXText']"
            ).text
            date_text = raw.replace("·","").replace(" ","")
        except Exception:
            date_text = ""

        time.sleep(random.uniform(2.5,3.5))

        # Description extraction
        try:
            raw = driver.find_element(
            By.CSS_SELECTOR,
            "div[class*='DivCustomTDKContainer']"
            ).text

            ia_disclaimer = "Esta información se generó por IA y puede presentar resultados que no son relevantes. No representa las opiniones o consejos de TikTok. Si tienes alguna duda, envíanosla a través de: Comentarios y ayuda: TikTok"

            desc_text = raw.replace(ia_disclaimer,"")

        except Exception:
            desc_text = ""

        time.sleep(random.uniform(0.5,1.5))


        # Get Locations 

         
        try:

            # Get address label

            # From caption 
            if d !='':
                excat_add =  wm.extract_addresses(d, top_k=1, min_score=2.5)
                if len(excat_add) > 0:
                    add_label =  excat_add[0].text
                else:
                    if len(wm.detect_places(d))>0:
                        add_label =  wm.detect_places(d)[0].text
                    else:
                        add_label = ''

            # Fromn text description 
            else:
                if desc_text != "":
                    excat_add =  wm.extract_addresses(desc_text, top_k=1, min_score=2.5)
                    if len(excat_add) > 0:
                        add_label =  excat_add[0].text
                    else:
                        if len(wm.detect_places(desc_text))>0:
                            add_label =  wm.detect_places(d)[0].text
                        else:
                            add_label = ''

                else:
                    add_label = ""

            # Call API if we have label
            if add_label != "":

                loc_osm_api = wm.geocode_osm(add_label)
                
                # Update interest values
                if loc_osm_api["found"]:
                    ind_found = True
                    lat = loc_osm_api["lat"]
                    lon = loc_osm_api["lon"]
                    osm_name_label = loc_osm_api["display_name"]
                    

                else:
                    ind_found = False
                    lat = 0.0
                    lon = 0.0
                    osm_name_label = ""

            # Empty values 
            else: 
                ### [COMPLETE WITH SPEECH TO TEXT OR VIDEO TO TEXT]

                ind_found = False
                lat = 0.0
                lon = 0.0
                osm_name_label = ""

            print(f"Geolocation Process ended for {h}")
                
        # Empty values 
        except:


            ind_found = False
            lat = 0.0
            lon = 0.0
            osm_name_label = ""
            print(f"Geolocation Process Failed for {h}")

        # Update results 
        desc_results.append({
                    "href": h,
                    "long_description": desc_text,
                    "date_long": date_text, 
                    "add_label_used": add_label
                    "ind_found":ind_found, 
                    "lat":lat,
                    "lon":lon, 
                    "osm_name_label":osm_name_label
                                            })

        time.sleep(random.uniform(0.5,1.5))

    # Improve with finally

        driver.quit()

    except:
        driver.quit()

Geolocation Process Failed for https://www.tiktok.com/@moniksmr/video/7545214454246444306
Geolocation Process ended for https://www.tiktok.com/@soymiauoficial/video/7416159212868259078
Geolocation Process ended for https://www.tiktok.com/@sally_qiiu/video/7591718836726975765
Geolocation Process ended for https://www.tiktok.com/@falvindates/video/7589317902264700180
Geolocation Process ended for https://www.tiktok.com/@moniksmr/video/7537785749467401490
Geolocation Process ended for https://www.tiktok.com/@nylagame/video/7591132225698090260
Geolocation Process ended for https://www.tiktok.com/@nylagame/video/7591395230469934357
Geolocation Process ended for https://www.tiktok.com/@nylagame/video/7591679944363740437
Geolocation Process ended for https://www.tiktok.com/@undiacondya/video/7515511709621079314
Geolocation Process ended for https://www.tiktok.com/@moniksmr/video/7572281843647630610
Geolocation Process ended for https://www.tiktok.com/@cocodidany/video/7590571924519013653
Geol

Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json)
Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json)
The chromedriver version (143.0.7499.40) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (); currently, chromedriver  is recommended for chrome .*, so it is advised to delete the driver in PATH and retry
There was an error managing chrome; using browser found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json)
Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chr

In [10]:
df_raw_des = pd.DataFrame(desc_results)

In [11]:
df_raw_des.to_csv(RAW_DATA + DAY_FILE, index = False)

In [18]:
df_raw_des.head(3)

,href,long_description,date_long,ind_found,lat,lon,osm_name_label
0,https://www.tiktok.com/@moniksmr/video/7545214...,Cafeterías Bonitas en la CDMX: Guía para Desay...,2025-9-1,False,0.000000,0.000000,NaN
1,https://www.tiktok.com/@soymiauoficial/video/7...,Parque San Nicolás Totolapan: Un Tesoro en CDM...,2024-9-18,True,19.418492,-99.157634,"Felix, 64, Avenida Álvaro Obregón, Roma Norte,..."
2,https://www.tiktok.com/@sally_qiiu/video/75917...,NaN,Hace13h,False,0.000000,0.000000,NaN


In [12]:
len(df_raw_des)

418

Date and description cleaning 

In [13]:
df_raw = pd.read_csv(RAW_DATA + ROOT_FILE)
df_raw_des = pd.read_csv(RAW_DATA + DAY_FILE)

In [14]:
df = (df_raw.merge(
    df_raw_des
    , on = 'href', how = 'left'
    )
    .dropna(subset=['date_long'])
)

In [17]:
df.head(5)

,href,description,date,hashtag_label,level,long_description,date_long,ind_found,lat,lon,osm_name_label
0,https://www.tiktok.com/@moniksmr/video/7545214...,NaN,NaN,#dondeircdmx,0,Cafeterías Bonitas en la CDMX: Guía para Desay...,2025-9-1,False,0.000000,0.000000,NaN
1,https://www.tiktok.com/@soymiauoficial/video/7...,NaN,NaN,#dondeircdmx,0,Parque San Nicolás Totolapan: Un Tesoro en CDM...,2024-9-18,True,19.418492,-99.157634,"Felix, 64, Avenida Álvaro Obregón, Roma Norte,..."
2,https://www.tiktok.com/@sally_qiiu/video/75917...,NaN,NaN,#dondeircdmx,0,NaN,Hace13h,False,0.000000,0.000000,NaN
3,https://www.tiktok.com/@falvindates/video/7589...,NaN,NaN,#dondeircdmx,0,Disfruta de un día en Acuario Michin en CDMX\n...,2025-12-29,True,19.418492,-99.157634,"Felix, 64, Avenida Álvaro Obregón, Roma Norte,..."
5,https://www.tiktok.com/@moniksmr/video/7537785...,NaN,NaN,#dondeircdmx,0,Explora el Parque Nacional Los Dinamos en CDMX...,2025-8-12,False,0.000000,0.000000,NaN


In [5]:
# Get full description 
df = df.query(""" long_description != '' """)

# Get data 
df.date_long = pd.to_datetime(df.date_long.apply(lambda x: 
                   pd.to_datetime(dt.datetime.today()).strftime('%Y-%m-%d') if 'Hace' in x else x)
                , format='%Y-%m-%d')

date_limit = pd.to_datetime(
    dt.datetime.today()- dt.timedelta(days=DAYS_FRESHNESS)
    ).strftime('%Y-%m-%d')


df = df.query(f""" date_long > '{date_limit}'""")

df = df[~(df.description.isna() & df.long_description.isna())]

df['description'] = df['description'].fillna('')
df['long_description'] = df['long_description'].fillna('')

In [6]:
def get_address(string):
    excat_add =  wm.extract_addresses(string, top_k=1, min_score=2.5)

    if len(excat_add) > 0:
        return wm.extract_addresses(string, top_k=1, min_score=2.5)[0].text
    else:
        if len(wm.detect_places(string)):
            return wm.detect_places(string)[0].text
        else:
            return ''

In [7]:
df["add_long"] = df.long_description.apply(get_address)
df["add_desc"] = df.description.apply(get_address)

In [30]:
df[df.add_desc != '']

,href,description,date,hashtag_label,level,long_description,date_long,add_long,add_desc
91,https://www.tiktok.com/@rivardo24/video/752754...,"Visita el Acuario Michin, el acuario más grand...",2025-7-15,#dondeircdmx,0,"Visita el Acuario Michin en CDMX, el más grand...",2025-07-15,"el Acuario Michin en CDMX, el más grande de La...","el Acuario Michin, el acuario más grande de La..."
92,https://www.tiktok.com/@kar_martinezg/video/75...,¿Cuánto cuesta ir a Six Flags México en Christ...,2025-12-26,#dondeircdmx,0,Precios para disfrutar Christmas In The Park e...,2025-12-26,Six Flags México,Christmas In The Park
94,https://www.tiktok.com/@valpina.mx/video/75173...,Quieres un PLAN DIFERENTE este fin de semana e...,2025-6-18,#dondeircdmx,0,,2025-06-18,,CDMX
95,https://www.tiktok.com/@whattodoinmexicocity/v...,"🎄✨ Navidad mágica en Banjército, CDMX ✨🎄",2025-12-27,#dondeircdmx,0,"Navidad Mágica en Banjército, CDMX: Plan Famil...",2025-12-27,"Banjército, CDMX: Plan Familiar Ideal","Banjército, CDMX ✨🎄"
108,https://www.tiktok.com/@imgabycontre/video/753...,Conoces este lugar? Esta es la Privada Roja en...,2025-7-23,#lugarescdmx,0,,2025-07-23,,CDMX
...,...,...,...,...,...,...,...,...,...
1462,https://www.tiktok.com/@mitikahmall/video/7510...,Restaurantes escondidos en Mítikah,2025-5-30,#restaurantescdmx,0,Descubre Restaurantes Escondidos en Mítikah CD...,2025-05-30,Mítikah CDMX,Mítikah
1467,https://www.tiktok.com/@sisomosgemelos/video/7...,Un buen Restaurante Italiano en CDMX 🍝🍕🇮🇹 📍4 M...,2025-7-4,#restaurantescdmx,0,Descubre 4 Mori: Un Restaurante Italiano en CD...,2025-07-04,CDMX,"📍4 Mori, 3900 Vasco de Quiroga (Sante Fe)"
1474,https://www.tiktok.com/@camaronbuchonmx/video/...,¿Mariscos buenos en CDMX? ¡Claro que si! En el...,2025-11-24,#restaurantescdmx,0,Los mejores mariscos en CDMX: Camarón Buchón\n...,2025-11-24,CDMX: Camarón Buchón,CDMX
1476,https://www.tiktok.com/@soyelarturito/video/75...,"Tacos Charly, CDMX! 🇲🇽🌮 Para muchos el mejor s...",2025-11-2,#restaurantescdmx,0,Tacos Charly: El Mejor Suadero de CDMX\nDescub...,2025-11-02,"CDMX, donde el suadero se convierte en una exp...",Tacos Charly | CDMX


In [23]:
df.loc[1467,'description']

'Un buen Restaurante Italiano en CDMX 🍝🍕🇮🇹 📍4 Mori, 3900 Vasco de Quiroga (Sante Fe)'

In [27]:
wm.extract_addresses(df.loc[1467,'description'], top_k=3, min_score=2.5)

[AddressMatch(text='📍4 Mori, 3900 Vasco de Quiroga (Sante Fe)', span=(42, 83), score=4.0, rule='emoji_soft')]

In [ ]:
df.loc[1467,'href']

'https://www.tiktok.com/@ely_guia/video/7587190357058735378'